<a href="https://colab.research.google.com/github/Daylon7/DS-Homeworks/blob/main/Tarea_Hypertuning_Parametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimizacion por Hiperparametros

---



## Previa

### Importacion de Datos

In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
# Establecer ruta de acceso en dr
import os
print(os.getcwd())
#os.chdir("")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content


Seleccion de Librerias

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import HalvingRandomSearchCV

**Datos**

https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud (El enlace tenia una "y" al final por error)


**Contexto**

Es importante que las compañías de tarjetas de crédito puedan reconocer las transacciones de tarjetas de crédito fraudulentas para que a los clientes no se les cobre por artículos que no compraron.

**Contenido**

El conjunto de datos contiene transacciones realizadas con tarjetas de crédito en septiembre de 2013 por titulares de tarjetas europeos.

Este conjunto de datos presenta transacciones que ocurrieron en dos días, donde tenemos 492 fraudes de 284,807 transacciones. El conjunto de datos está muy desequilibrado, la clase positiva (fraudes) representa el 0,172 % de todas las transacciones.

Contiene solo variables de entrada numéricas que son el resultado de una transformación PCA.

Desafortunadamente, debido a problemas de confidencialidad, no se pueden proporcionar las características originales ni más información general sobre los datos. Las características V1, V2, … V28 son los principales componentes obtenidos con PCA, las únicas características que no han sido transformadas con PCA son **Time y Amount**.


La característica **Time** contiene los segundos transcurridos entre cada transacción y la primera transacción en el conjunto de datos. La variable **Amount** es la cantidad de la transacción, esta función se puede utilizar para el aprendizaje sensible a los costos dependiente del ejemplo. **Class** es la variable de respuesta y toma valor 1 en caso de fraude y 0 en caso contrario.

Informacion adicional del enlace: Dada la relación de desequilibrio de clases, recomendamos medir la precisión utilizando el Area Under the Precision-Recall Curve (AUPRC). La precisión de la matriz de confusión no es significativa para una clasificación

---



El dataset esta muy desbalanceado, es decir, de 284,807 transacciones solo hay 492 que estan clasificados como fraude. Y se nos da como dato que la clase positiva (fraudes) representa el 0,172 % de todas las transacciones.

En ese sentido, se realiza una muestra de la siguiente forma

In [3]:
# Lectura de DF
df=pd.read_csv('/content/gdrive/My Drive/creditcard.csv')
df.head(5)
# Vamos a eliminar la columna tiempo
df= df.drop(columns='Time')
# Estandarizamos la columna Amount
df['Amount']=(df['Amount']- np.mean(df['Amount']))/np.std(df.Amount) # Valor Estandarizado = (Monto Original - Media)/Desviacion Estandar
df.head()
# El problema es muy desbalanceado tranajaremos con una muestra para resolver el problema
df_ones=df[df['Class']==1] # Filtro de caracteristica. Se extraen los fraudes
print(df_ones.shape)
df_zeros=df[df['Class']==0] # Filtro de NO caracteristica.  Se extraen los que no son fraudes
df_zeros= df_zeros.sample(3*df_ones.shape[0]) # Tamaño de muestra 3 veces el de la caracteristica. Sample hace una muestra aleatoria del conjunto. shape[0] significa que solo va a tomar en cuenta el numero de filas
print(df_zeros.shape) #Verificamos
# Concatenar
df_final=pd.DataFrame(np.concatenate([df_ones, df_zeros],axis=0), columns=df.columns) #Unimos ambos dataframes
print(df_final.shape)
df_final.head() #Verificamos

(492, 30)
(1476, 30)
(1968, 30)


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,-2.772272,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,-0.353229,1.0
1,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,-0.838587,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,1.761758,1.0
2,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,-1.525412,...,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,0.606031,1.0
3,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,-4.801637,...,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,-0.117342,1.0
4,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,-2.447469,...,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,-0.349231,1.0


In [4]:
df_final.shape

(1968, 30)

In [5]:
df_final.isnull().sum()

,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0
V10,0


Como se puede observar, el dataset no posee nulos que puedan perjudicar el analisis posterior.

---



In [6]:
# Separar en X y y
y= df_final.Class
X= df_final.drop(columns='Class', axis=1)
print(X.shape, y.shape)

(1968, 29) (1968,)


In [7]:
# Separar en train y test
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape, X_test.shape)

(1377, 29) (591, 29)


# Ejemplo de Seleccion manual

In [8]:
model= xgb.XGBClassifier(learning_rate=0.001)
# Lista de hiperparametros
params_1 = {'criterion': 'gini', 'splitter': 'best', 'max_depth': 5}
params_2 = {'criterion': 'entropy', 'splitter': 'random', 'max_depth': 7}
params_3 = {'criterion': 'gini', 'splitter': 'random', 'max_depth': 10}

## Estos procesos pueden demorar bastante !!

In [9]:
# Modelo 1
model.set_params(**params_1).fit(X_train, y_train)
print(f'Accuracy para Modelo 1 = {round(accuracy_score(y_test, model.predict(X_test)), 5)}')
# Modelo 2
model.set_params(**params_2).fit(X_train, y_train)
print(f'Accuracy para Modelo 2 = {round(accuracy_score(y_test, model.predict(X_test)), 5)}')
# Modelo 3
model.set_params(**params_3).fit(X_train, y_train)
print(f'Accuracy para Modelo 3 = {round(accuracy_score(y_test, model.predict(X_test)), 5)}')

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:52:46] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "criterion", "splitter" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy para Modelo 1 = 0.74619


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:52:47] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "criterion", "splitter" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy para Modelo 2 = 0.74619


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:52:47] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "criterion", "splitter" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy para Modelo 3 = 0.74619


# Grid Search

In [10]:
params_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [5,6,7],
        'criterion':['entropy','gini']
        }

In [11]:
3*4*3*3*3*2

648

In [21]:
#Se tuvo que usar una version de Sk-learn mas antigua para que Grid Search funcionara (1.3.1). Arroja el siguiente error ='super' object has no attribute '__sklearn_tags__'.
#El resto de los modelos funciona con la version mas reciente
#!pip uninstall -y scikit-learn
!pip install scikit-learn==1.3.1 #Colocar aqui 1.3.1 en caso de querer usar Grid Search

  Using cached scikit_learn-1.3.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.3.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.9 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.0
    Uninstalling scikit-learn-1.6.0:
      Successfully uninstalled scikit-learn-1.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.3.1 which is incompatible.


In [17]:
# tiempo de ejecucuon 424 seg
grid_cv = GridSearchCV(model, params_grid, scoring="accuracy", n_jobs=-1, cv=3)
grid_cv.fit(X_train, y_train)

print("Mejores Parametros", grid_cv.best_params_)
print("Mejor CV score", grid_cv.best_score_)
print(f'Accuracy del modelo = {round(accuracy_score(y_test, grid_cv.predict(X_test)), 5)}')

AttributeError: 'super' object has no attribute '__sklearn_tags__'

In [ ]:
pd.DataFrame( grid_cv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_criterion,param_gamma,param_max_depth,param_min_child_weight,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.866463,0.148453,0.052388,0.013931,0.6,entropy,0.5,5,1,0.6,"{'colsample_bytree': 0.6, 'criterion': 'entrop...",0.751634,0.751634,0.751634,0.751634,0.0,1
1,0.908889,0.179827,0.043583,0.015887,0.6,entropy,0.5,5,1,0.8,"{'colsample_bytree': 0.6, 'criterion': 'entrop...",0.751634,0.751634,0.751634,0.751634,0.0,1
2,1.003148,0.104060,0.055506,0.006230,0.6,entropy,0.5,5,1,1.0,"{'colsample_bytree': 0.6, 'criterion': 'entrop...",0.751634,0.751634,0.751634,0.751634,0.0,1
3,0.549970,0.049527,0.043818,0.022364,0.6,entropy,0.5,5,5,0.6,"{'colsample_bytree': 0.6, 'criterion': 'entrop...",0.751634,0.751634,0.751634,0.751634,0.0,1
4,0.621240,0.100277,0.056937,0.009803,0.6,entropy,0.5,5,5,0.8,"{'colsample_bytree': 0.6, 'criterion': 'entrop...",0.751634,0.751634,0.751634,0.751634,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,0.244074,0.011602,0.015904,0.001355,1.0,gini,5,7,5,0.8,"{'colsample_bytree': 1.0, 'criterion': 'gini',...",0.751634,0.751634,0.751634,0.751634,0.0,1
806,0.262650,0.027539,0.015715,0.000764,1.0,gini,5,7,5,1.0,"{'colsample_bytree': 1.0, 'criterion': 'gini',...",0.751634,0.751634,0.751634,0.751634,0.0,1
807,0.164908,0.003896,0.018312,0.001153,1.0,gini,5,7,10,0.6,"{'colsample_bytree': 1.0, 'criterion': 'gini',...",0.751634,0.751634,0.751634,0.751634,0.0,1
808,0.194703,0.010594,0.018556,0.005054,1.0,gini,5,7,10,0.8,"{'colsample_bytree': 1.0, 'criterion': 'gini',...",0.751634,0.751634,0.751634,0.751634,0.0,1


# Randomized Search CV

Usaremos este modelo debido a su rapidez en la ejecucion

In [12]:
# tiempo de ejecucuon 7 seg
grid_cv = RandomizedSearchCV(model, params_grid, scoring="accuracy", n_jobs=-1, cv=3)
grid_cv.fit(X_train, y_train)

print("Mejores parametros", grid_cv.best_params_)
print("Mejor score de CV", grid_cv.best_score_)
print(f'Accuracy del modelo = {round(accuracy_score(y_test, grid_cv.predict(X_test)), 5)}')

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:53:14] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "criterion", "splitter" } are not used.

  warnings.warn(smsg, UserWarning)


Mejores parametros {'subsample': 1.0, 'min_child_weight': 5, 'max_depth': 7, 'gamma': 0.5, 'criterion': 'gini', 'colsample_bytree': 0.6}
Mejor score de CV 0.7516339869281046
Accuracy del modelo = 0.74619


# Halving Grid Search

In [19]:
# tiempo de ejecucuon 125 seg
halving_cv = HalvingGridSearchCV(model, params_grid, scoring="accuracy", factor=3)
halving_cv.fit(X_train, y_train)

print("Mejores parametros", halving_cv.best_params_)
print("Mejor Score CV", halving_cv.best_score_)
print(f'Accuracy del modelo = {round(accuracy_score(y_test, halving_cv.predict(X_test)), 5)}')

AttributeError: 'super' object has no attribute '__sklearn_tags__'

# Halving Randomized Search

In [20]:
# tiempo de ejecucuon 13 seg
halving_cv = HalvingRandomSearchCV(model, params_grid, scoring="accuracy", factor=3)
halving_cv.fit(X_train, y_train)

print("Mejores parametros", halving_cv.best_params_)
print("Mejor CV score", halving_cv.best_score_)
print(f'Accuracy del modelo = {round(accuracy_score(y_test, halving_cv.predict(X_test)), 5)}')

AttributeError: 'super' object has no attribute '__sklearn_tags__'

# Random Forest

Para el random forest se tomo en cuenta el siguiente articulo de consulta para la definicion de los hiperparametros

https://panamahitek.com/optimizacion-de-hiperparametros-machine-learning/

Mas informacion acerca de entropia y el coeficiente de gini https://quantdare.com/decision-trees-gini-vs-entropy/

In [13]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
params_grid = {
        'n_estimators': [10, 100, 200], # El número de árboles en el bosque.
        'max_depth': [3, 5, 7, 10, 15], # La profundidad máxima de cada árbol en el bosque
        'min_samples_split': [2, 5, 10], #El número mínimo de muestras necesarias para dividir un nodo interno.
        'criterion':['entropy','gini']
        }
grid_cv = RandomizedSearchCV(clf, params_grid, scoring="accuracy", n_jobs=-1, cv=3) # Usamos este modelo por su rapidez a la hora de la ejecucion
grid_cv.fit(X_train, y_train)

print("Mejores parametros", grid_cv.best_params_)
print("Mejor score de CV", grid_cv.best_score_)
print(f'Accuracy del modelo = {round(accuracy_score(y_test, grid_cv.predict(X_test)), 5)}')

Mejores parametros {'n_estimators': 10, 'min_samples_split': 5, 'max_depth': 7, 'criterion': 'entropy'}
Mejor score de CV 0.9607843137254902
Accuracy del modelo = 0.96785


# KNN

Para la eleccion de los hiperparametros se tomo en cuenta el siguiente articulo https://www.kaggle.com/code/arunimsamudra/k-nn-with-hyperparameter-tuning

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

neigh = KNeighborsClassifier() #instanciar el objeto

params_grid = { 'n_neighbors' : [3,4,5,6,7,9,11,13,15], # Numero de vecinos
               'weights' : ['uniform','distance'],  # Uniforme se refiere a que todos los puntos de un grupo tienen el mismo peso, distancia es cuando toma relevancia la cercania entre un punto y otro del grupo
               'metric' : ['minkowski','euclidean','manhattan']} # Tipo de distancia

grid_cv = RandomizedSearchCV(neigh, params_grid, scoring="accuracy", n_jobs=-1, cv=3) # Usamos este modelo por su rapidez a la hora de la ejecucion
grid_cv.fit(X_train, y_train)

print("Mejores parametros", grid_cv.best_params_)
print("Mejor score de CV", grid_cv.best_score_)
print(f'Accuracy del modelo = {round(accuracy_score(y_test, grid_cv.predict(X_test)), 5)}')

Mejores parametros {'weights': 'distance', 'n_neighbors': 6, 'metric': 'minkowski'}
Mejor score de CV 0.963689179375454
Accuracy del modelo = 0.96447


En este caso, se puede ver que la distancia es el valor adecuado para el parametro Weigths, el numero de los vecinos optimo es 6 de la eleccion presente, y el parametro metric que hace referencia al tipo de distancia se tiene que el optimo es del tipo Minkowski



# Regresion Logistica

Se tomo en cuenta los siguientes articulos:

https://scikit-learn.org/1.5/modules/generated/sklearn.linear_model.LogisticRegression.html

https://www.kaggle.com/code/funxexcel/p2-logistic-regression-hyperparameter-tuning

In [19]:
from sklearn.linear_model import LogisticRegression
logModel = LogisticRegression()

params_grid = {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : [100, 1000,2500, 5000]}    #Codigo Extraido del ejemplo de Kaggle

grid_cv = RandomizedSearchCV(logModel, params_grid, scoring="accuracy", n_jobs=-1, cv=3) # Usamos este modelo por su rapidez a la hora de la ejecucion
grid_cv.fit(X_train, y_train)

print("Mejores parametros", grid_cv.best_params_)
print("Mejor score de CV", grid_cv.best_score_)
print(f'Accuracy del modelo = {round(accuracy_score(y_test, grid_cv.predict(X_test)), 5)}')


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
9 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py", line 1179, in fit
    raise ValueError("l1_rat

Mejores parametros {'solver': 'saga', 'penalty': 'l1', 'max_iter': 1000, 'C': 1.623776739188721}
Mejor score de CV 0.9600580973129994
Accuracy del modelo = 0.95939


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


En general el uso de los tres modelos anteriores pueden recomendarse ya que tienen similar accuracy
Sin embargdo, de escogerse un modelo, el que tiene mejor Accuracy es el Random Forest con n_estimators': 10, 'min_samples_split': 5, 'max_depth': 7, 'criterion': 'entropy'

